In [4]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests
from torchvision import datasets, transforms
import json

# https://github.com/huggingface/transformers/blob/main/src/transformers/models/detr/modeling_detr.py

In [5]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

c:\Users\agama\anaconda3\envs\mlops\lib\site-packages\transformers\models\detr\image_processing_detr.py:773: FutureWarning: The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
  warnings.warn(


In [19]:
print(outputs) 

DetrObjectDetectionOutput(loss=None, loss_dict=None, logits=tensor([[[-19.1194,  -0.0893, -11.0154,  ...,  -8.7117,  -9.8543,  10.6531],
         [-17.3640,  -1.8035, -14.0219,  ...,  -8.2177,  -4.1631,  11.4021],
         [-20.0461,  -0.5837, -11.1060,  ...,  -9.2851, -12.1836,  11.1371],
         ...,
         [-19.7237,  -1.3058, -10.1437,  ..., -10.4849, -12.9238,  11.2945],
         [-12.8113,   1.9661,  -5.0197,  ...,  -6.7028,  -7.3127,   6.2644],
         [-17.3271,  -0.8608, -13.1640,  ...,  -6.6456,  -3.8822,  10.6797]]],
       grad_fn=<ViewBackward0>), pred_boxes=tensor([[[0.4433, 0.5302, 0.8853, 0.9056],
         [0.5494, 0.2517, 0.0529, 0.2015],
         [0.4998, 0.5360, 0.9956, 0.9024],
         [0.5534, 0.1835, 0.0415, 0.0616],
         [0.4998, 0.4116, 0.9948, 0.8237],
         [0.5957, 0.5033, 0.8091, 0.9647],
         [0.5416, 0.2791, 0.0393, 0.2099],
         [0.1607, 0.2160, 0.2274, 0.1434],
         [0.1738, 0.1822, 0.1956, 0.0785],
         [0.4997, 0.3887, 0.999

In [20]:
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

In [21]:
results['boxes']

{'scores': tensor([0.9982, 0.9960, 0.9955, 0.9988, 0.9987], grad_fn=<IndexBackward0>),
 'labels': tensor([75, 75, 63, 17, 17]),
 'boxes': tensor([[ 4.0163e+01,  7.0812e+01,  1.7555e+02,  1.1798e+02],
         [ 3.3324e+02,  7.2550e+01,  3.6833e+02,  1.8766e+02],
         [-2.2602e-02,  1.1496e+00,  6.3973e+02,  4.7376e+02],
         [ 1.3241e+01,  5.2055e+01,  3.1402e+02,  4.7093e+02],
         [ 3.4540e+02,  2.3854e+01,  6.4037e+02,  3.6872e+02]],
        grad_fn=<IndexBackward0>)}

In [2]:
f = open('./annotations/instances_val2017.json')
instances = json.load(f)
instances.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [3]:
# getting annotations for exemplary validation file we have
for i in range(len(instances['annotations'])):
    if instances['annotations'][i]['image_id'] == 39769:
        print(instances['annotations'][i]['bbox'])
        print(instances['annotations'][i]['category_id'])

[332.89, 79.57, 38.65, 106.31]
75
[41.88, 74.2, 133.15, 45.1]
75
[1.08, 0.0, 638.56, 473.53]
63
[1.07, 1.18, 638.93, 477.85]
65
[17.5, 54.38, 301.25, 415.0]
17
[347.5, 25.63, 292.5, 343.75]
17


In [8]:
# creating the labels dictionary needed for training - should be list of Dicts ? (for a whole batch of images)
dict = {'class_labels': torch.LongTensor([75,75,63,65,17,17]),
'boxes':torch.FloatTensor([[332.89, 79.57, 38.65, 106.31],
[41.88, 74.2, 133.15, 45.1],
[1.08, 0.0, 638.56, 473.53],
[1.07, 1.18, 638.93, 477.85],
[17.5, 54.38, 301.25, 415.0],
[347.5, 25.63, 292.5, 343.75]])}

In [10]:
dict

{'class_labels': tensor([75, 75, 63, 65, 17, 17]),
 'boxes': tensor([[332.8900,  79.5700,  38.6500, 106.3100],
         [ 41.8800,  74.2000, 133.1500,  45.1000],
         [  1.0800,   0.0000, 638.5600, 473.5300],
         [  1.0700,   1.1800, 638.9300, 477.8500],
         [ 17.5000,  54.3800, 301.2500, 415.0000],
         [347.5000,  25.6300, 292.5000, 343.7500]])}

In [11]:
model(image, labels=[dict])

AttributeError: shape